# Architecture 1: Self-Reconstruction Multi-Modal Autoencoder

This notebook implements the original architecture where:
- Image features are reconstructed by the image autoencoder
- Text features are reconstructed by the text autoencoder
- Latent spaces are aligned using contrastive loss

**Prerequisites:** Run `Feature_Extraction_Batch.ipynb` first to generate the required .npy files.

5. Multi-Modal Autoencoder Architecture

In [37]:
# Now time to set up and train the actual multi-modal autoencoder
config = {
    "image_feat_path": "train_image_features.npy",   # we'll load proper files below
    "caption_feat_path": "train_caption_features.npy",
    "caption_to_image_path": "train_caption_to_image.npy",
    "val_image_feat_path": "val_image_features.npy",
    "val_caption_feat_path": "val_caption_features.npy",
    "val_caption_to_image_path": "val_caption_to_image.npy",
    "latent_dim": 512,
    "img_input_dim": 2048,
    "txt_input_dim": 768,
    "img_hidden": 1024,
    "txt_hidden": 512,
    "batch_size": 128,    # try 128; lower if memory limited (e.g., 64)
    "lr": 1e-3,
    "weight_decay": 1e-5,
    "epochs": 40,
    "lambda_align": 1.0,  # weight for latent alignment loss; tuneable
    "checkpoint_dir": "./corr_ae_checkpoints_contrastive",
    "seed": 42,
}

In [38]:
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])

os.makedirs(config["checkpoint_dir"], exist_ok=True)

In [39]:
#I don't need to reload these if we run it all in the same notebook but I'm pasting the load functions here anyway
image_train = np.load(config["image_feat_path"])
caption_train = np.load(config["caption_feat_path"])
cap2img_train = np.load(config["caption_to_image_path"])

image_val = np.load(config["val_image_feat_path"])
caption_val = np.load(config["val_caption_feat_path"])
cap2img_val = np.load(config["val_caption_to_image_path"])

print("Shapes (train):", image_train.shape, caption_train.shape, cap2img_train.shape)
print("Shapes (val):", image_val.shape, caption_val.shape, cap2img_val.shape)

Shapes (train): (5663, 2048) (28315, 768) (28315,)
Shapes (val): (1214, 2048) (6070, 768) (6070,)


In [40]:
# 3) Compute train-set normalization (mean/std) and apply to all splits
# Normalize per-feature (column-wise) using training set statistics
img_mean = image_train.mean(axis=0, keepdims=True)
img_std = image_train.std(axis=0, keepdims=True) + 1e-6

txt_mean = caption_train.mean(axis=0, keepdims=True)
txt_std = caption_train.std(axis=0, keepdims=True) + 1e-6

In [41]:
def normalize_images(x):
    return (x - img_mean) / img_std

def normalize_texts(x):
    return (x - txt_mean) / txt_std

image_train = normalize_images(image_train)
image_val   = normalize_images(image_val)

caption_train = normalize_texts(caption_train)
caption_val   = normalize_texts(caption_val)

In [42]:
# 4) Dataset that returns paired (image_feat, caption_feat) for each caption
class CaptionImagePairedDataset(Dataset):
    """
    Iterates over captions. For index i, returns:
      caption_features[i], image_features[ caption_to_image_idx[i] ]
    """
    def __init__(self, caption_feats, image_feats, caption_to_image_idx):
        assert len(caption_feats) == len(caption_to_image_idx)
        self.caption_feats = caption_feats.astype(np.float32)
        self.image_feats = image_feats.astype(np.float32)
        self.cap2img = caption_to_image_idx.astype(np.int64)

    def __len__(self):
        return len(self.caption_feats)

    def __getitem__(self, idx):
        cap = self.caption_feats[idx]
        img = self.image_feats[self.cap2img[idx]]
        return {"image": torch.from_numpy(img), "caption": torch.from_numpy(cap)}

train_dataset = CaptionImagePairedDataset(caption_train, image_train, cap2img_train)
val_dataset   = CaptionImagePairedDataset(caption_val, image_val, cap2img_val)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=0)

In [ ]:
# 5) Model: two autoencoders with shared latent dimension
class ImageAE(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=1024, latent_dim=512):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

class TextAE(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=512, latent_dim=512):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

In [44]:
# contrastive loss function
def contrastive_loss(z_img, z_txt, temperature=0.07):
    # normalize both embeddings
    z_img_norm = F.normalize(z_img, dim=1)
    z_txt_norm = F.normalize(z_txt, dim=1)

    # compute similarity matrix (batch_size x batch_size )
    logits = torch.matmul(z_img_norm, z_txt_norm.T) / temperature

    # create labels
    labels = torch.arange(z_img.size(0)).to(z_img.device)

    # compute cross-entropy for image to text direction
    loss_i2t = F.cross_entropy(logits, labels)

    # compute cross-entropy for text-to-image direction
    loss_t2i = F.cross_entropy(logits.T, labels)
    
    # average the two losses
    return (loss_i2t + loss_t2i) /2

In [45]:
# Instantiate models
img_ae = ImageAE(
    input_dim=config["img_input_dim"],
    hidden_dim=config["img_hidden"],
    latent_dim=config["latent_dim"]
).to(device)

txt_ae = TextAE(
    input_dim=config["txt_input_dim"],
    hidden_dim=config["txt_hidden"],
    latent_dim=config["latent_dim"]
).to(device)

# 6) Losses and optimizer
recon_loss_fn = nn.MSELoss()    # reconstruction for both
# align_loss_fn = nn.MSELoss()    # align latents

params = list(img_ae.parameters()) + list(txt_ae.parameters())
optimizer = Adam(params, lr=config["lr"], weight_decay=config["weight_decay"])

6. Training

In [ ]:
# 7) Training / validation loop
def run_epoch(loader, training=True):
    if training:
        img_ae.train(); txt_ae.train()
    else:
        img_ae.eval(); txt_ae.eval()

    total_recon_img = 0.0
    total_recon_txt = 0.0
    total_align = 0.0
    total_loss = 0.0
    n_samples = 0

    pbar = tqdm(loader, desc="train" if training else "val")
    with torch.set_grad_enabled(training):
        for batch in pbar:
            imgs = batch["image"].to(device)    # shape (B, img_dim)
            caps = batch["caption"].to(device)  # shape (B, txt_dim)
            batch_size = imgs.shape[0]

            # forward
            z_img, img_recon = img_ae(imgs)
            z_txt, txt_recon = txt_ae(caps)

            # losses
            L_img = recon_loss_fn(img_recon, imgs)
            L_txt = recon_loss_fn(txt_recon, caps)
            # L_align = align_loss_fn(z_img, z_txt)
            L_align = contrastive_loss(z_img, z_txt, temperature=0.7)

            loss = L_img + L_txt + config["lambda_align"] * L_align

            if training:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_recon_img += L_img.item() * batch_size
            total_recon_txt += L_txt.item() * batch_size
            total_align += L_align.item() * batch_size
            total_loss += loss.item() * batch_size
            n_samples += batch_size

            pbar.set_postfix({
                "loss": f"{total_loss / n_samples:.4f}",
                "Limg": f"{total_recon_img / n_samples:.4f}",
                "Ltxt": f"{total_recon_txt / n_samples:.4f}",
                "Lalign": f"{total_align / n_samples:.4f}"
            })

    return {
        "loss": total_loss / n_samples,
        "Limg": total_recon_img / n_samples,
        "Ltxt": total_recon_txt / n_samples,
        "Lalign": total_align / n_samples
    }

In [47]:
best_val_loss = float("inf")

for epoch in range(1, config["epochs"] + 1):
    print(f"\n=== Epoch {epoch}/{config['epochs']} ===")
    train_metrics = run_epoch(train_loader, training=True)
    val_metrics = run_epoch(val_loader, training=False)

    print(f"Train loss: {train_metrics['loss']:.4f} | Val loss: {val_metrics['loss']:.4f}")

    # Save checkpoint (every epoch)
    ckpt = {
        "epoch": epoch,
        "img_state": img_ae.state_dict(),
        "txt_state": txt_ae.state_dict(),
        "optimizer": optimizer.state_dict(),
        "train_metrics": train_metrics,
        "val_metrics": val_metrics,
        "config": config
    }
    ckpt_path = os.path.join(config["checkpoint_dir"], f"corr_ae_epoch{epoch}.pt")
    torch.save(ckpt, ckpt_path)

    # Keep best
    if val_metrics["loss"] < best_val_loss:
        best_val_loss = val_metrics["loss"]
        torch.save(ckpt, os.path.join(config["checkpoint_dir"], "corr_ae_best.pt"))
        print("Saved best checkpoint.")

print("Training finished.")


=== Epoch 1/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 52.04it/s, loss=4.6222, Limg=0.3231, Ltxt=0.2561, Lalign=4.0430]


Train loss: 4.8599 | Val loss: 4.6222
Saved best checkpoint.

=== Epoch 2/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 60.32it/s, loss=4.5043, Limg=0.2619, Ltxt=0.2119, Lalign=4.0305]


Train loss: 4.3459 | Val loss: 4.5043
Saved best checkpoint.

=== Epoch 3/40 ===


...

Train loss: 3.8536 | Val loss: 4.3176

=== Epoch 39/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 55.38it/s, loss=4.3109, Limg=0.1992, Ltxt=0.0868, Lalign=4.0248]


Train loss: 3.8513 | Val loss: 4.3109
Saved best checkpoint.

=== Epoch 40/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 60.50it/s, loss=4.3101, Limg=0.2027, Ltxt=0.0871, Lalign=4.0203]


Train loss: 3.8497 | Val loss: 4.3101
Saved best checkpoint.
Training finished.


7. Evaluation

In [48]:
#Now time to evaluate on the validation set
#Probably don't need to reload the model, but I'm going to include the code again in case we break this up into more managable files
# --- Load best checkpoint ---
best_ckpt_path = os.path.join(config["checkpoint_dir"], "corr_ae_best.pt")
ckpt = torch.load(best_ckpt_path, map_location=device)

img_ae.load_state_dict(ckpt["img_state"])
txt_ae.load_state_dict(ckpt["txt_state"])
img_ae.eval(); txt_ae.eval()

print(f"Loaded best checkpoint from {best_ckpt_path} (epoch {ckpt['epoch']})")

Loaded best checkpoint from ./corr_ae_checkpoints_contrastive/corr_ae_best.pt (epoch 40)


In [49]:
# Encode into latent space
with torch.no_grad():
    # Encode images
    Z_imgs = []
    for i in range(0, image_val.shape[0], 256):
        batch = torch.from_numpy(image_val[i:i+256]).float().to(device)
        z, _ = img_ae(batch)
        Z_imgs.append(z.cpu().numpy())
    Z_imgs = np.concatenate(Z_imgs, axis=0)   # shape (N_images, latent_dim)

    # Encode captions
    Z_caps = []
    for i in range(0, caption_val.shape[0], 256):
        batch = torch.from_numpy(caption_val[i:i+256]).float().to(device)
        z, _ = txt_ae(batch)
        Z_caps.append(z.cpu().numpy())
    Z_caps = np.concatenate(Z_caps, axis=0)   # shape (N_captions, latent_dim)

print("Encoded latent shapes:", Z_imgs.shape, Z_caps.shape)

Encoded latent shapes: (1214, 512) (6070, 512)


In [50]:
#Use Recall@1/5/10 to evaluate hyperparameter performance
#Note that we are using cosine similarity
#Should we consider using L2 metric instead? Does this even make sense?
def retrieval_metrics(Z_caps, Z_imgs, caption_to_image_idx):
    sims = cosine_similarity(Z_caps, Z_imgs)  # (num_caps, num_imgs)
    ranks = []
    for i, true_img_idx in enumerate(caption_to_image_idx):
        sim_scores = sims[i]
        sorted_indices = np.argsort(-sim_scores)  # descending
        rank = np.where(sorted_indices == true_img_idx)[0][0] + 1
        ranks.append(rank)

    ranks = np.array(ranks)
    recall_at_1  = np.mean(ranks <= 1)
    recall_at_5  = np.mean(ranks <= 5)
    recall_at_10 = np.mean(ranks <= 10)
    med_rank = np.median(ranks)

    return {
        "Recall@1": recall_at_1,
        "Recall@5": recall_at_5,
        "Recall@10": recall_at_10,
        "MedianRank": med_rank
    }

metrics_val = retrieval_metrics(Z_caps, Z_imgs, cap2img_val)
for k, v in metrics_val.items():
    print(f"{k}: {v:.4f}")

Recall@1: 0.0465
Recall@5: 0.1588
Recall@10: 0.2631
MedianRank: 30.0000


In [51]:
# evaluate on test set
# Encode into latent space
with torch.no_grad():
    # Encode images
    Z_imgs = []
    for i in range(0, image_test.shape[0], 256):
        batch = torch.from_numpy(image_test[i:i+256]).float().to(device)
        z, _ = img_ae(batch)
        Z_imgs.append(z.cpu().numpy())
    Z_imgs = np.concatenate(Z_imgs, axis=0)   # shape (N_images, latent_dim)

    # Encode captions
    Z_caps = []
    for i in range(0, caption_test.shape[0], 256):
        batch = torch.from_numpy(caption_test[i:i+256]).float().to(device)
        z, _ = txt_ae(batch)
        Z_caps.append(z.cpu().numpy())
    Z_caps = np.concatenate(Z_caps, axis=0)   # shape (N_captions, latent_dim)

print("Encoded latent shapes:", Z_imgs.shape, Z_caps.shape)

def retrieval_metrics(Z_caps, Z_imgs, caption_to_image_idx):
    sims = cosine_similarity(Z_caps, Z_imgs)  # (num_caps, num_imgs)
    ranks = []
    for i, true_img_idx in enumerate(caption_to_image_idx):
        sim_scores = sims[i]
        sorted_indices = np.argsort(-sim_scores)  # descending
        rank = np.where(sorted_indices == true_img_idx)[0][0] + 1
        ranks.append(rank)

    ranks = np.array(ranks)
    recall_at_1  = np.mean(ranks <= 1)
    recall_at_5  = np.mean(ranks <= 5)
    recall_at_10 = np.mean(ranks <= 10)
    med_rank = np.median(ranks)

    return {
        "Recall@1": recall_at_1,
        "Recall@5": recall_at_5,
        "Recall@10": recall_at_10,
        "MedianRank": med_rank
    }

metrics_val = retrieval_metrics(Z_caps, Z_imgs, cap2img_val)
for k, v in metrics_val.items():
    print(f"{k}: {v:.4f}")

Encoded latent shapes: (1214, 512) (6070, 512)
Recall@1: 0.0007
Recall@5: 0.0035
Recall@10: 0.0086
MedianRank: 600.0000


In [ ]:
# Prepare data for visualization: get actual caption texts and image filenames for validation set
# Extract the validation caption texts from the original dataframe
val_caption_texts = df["caption"].values[val_mask]

# Extract the validation image filenames
val_image_names = np.array([image_names[i] for i in val_idx])

print(f"Loaded {len(val_caption_texts)} caption texts and {len(val_image_names)} image filenames for validation")
print(f"Example caption: {val_caption_texts[0]}")
print(f"Example image: {val_image_names[0]}")

In [ ]:
#Quick visualization of what images are retrieved by what caption:
image_dir = "/Users/sfowler14/Downloads/archive/Images"  # Update this if your images are elsewhere

def show_top_images_for_caption(caption_idx, top_k=5):
    """
    Show top-k retrieved validation images for a given caption index.
    Also displays the true image for comparison.
    """
    # Get the embedding for this caption
    caption_embedding = Z_caps[caption_idx].reshape(1, -1)
    sims = cosine_similarity(caption_embedding, Z_imgs)[0]
    top_img_indices = np.argsort(-sims)[:top_k]

    # Print caption text
    print(f"\nCAPTION: {val_caption_texts[caption_idx]}")
    true_img_idx = cap2img_val[caption_idx]
    print(f"TRUE IMAGE: {val_image_names[true_img_idx]} (index {true_img_idx})")
    
    # Display: true image + top k retrieved images
    plt.figure(figsize=(18, 4))
    
    # Show the true image first
    true_img_name = val_image_names[true_img_idx]
    true_img_path = os.path.join(image_dir, true_img_name)
    try:
        true_img = Image.open(true_img_path)
        plt.subplot(1, top_k + 1, 1)
        plt.imshow(true_img)
        plt.axis('off')
        plt.title("TRUE IMAGE", fontweight='bold', color='green')
    except Exception as e:
        print(f"Could not open true image {true_img_path}: {e}")
    
    # Show retrieved images
    for i, img_idx in enumerate(top_img_indices):
        img_name = val_image_names[img_idx]
        img_path = os.path.join(image_dir, img_name)
        try:
            img = Image.open(img_path)
        except Exception as e:
            print(f"Could not open {img_path}: {e}")
            continue
        
        plt.subplot(1, top_k + 1, i + 2)
        plt.imshow(img)
        plt.axis('off')
        
        # Highlight if this retrieved image matches the true image
        if img_idx == true_img_idx:
            plt.title(f"Rank {i+1} ✓", fontweight='bold', color='green')
        else:
            plt.title(f"Rank {i+1}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
import random
for i in random.sample(range(len(caption_val)), 3):
    show_top_images_for_caption(i, top_k=5)